<a href="https://colab.research.google.com/github/NagarjunaD024/Datascience-LLMS/blob/main/src/Cost_optimizing_GPT/Basic_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openai
import pandas as pd
import time

from google.colab import drive, userdata,  output
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
client = openai.OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

In [ ]:
def create_prompt(text):
    """ Create prompt for sentiment classification.

    Args:
        text: text to classify.

    Returns:
        Prompt for text classification.
    """
    task = 'Is the sentiment positive or negative?'
    answer_format = 'Answer ("pos"/"neg")'
    return f'{text}\n{task}\n{answer_format}:'

In [ ]:
def call_llm(prompt):
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model and total number of tokens.
    """
    for nr_retries in range(1, 4):
        try:
            response = client.chat.completions.create(
                model='gpt-3.5-turbo',
                messages=[
                    {'role':'user', 'content':prompt}
                    ],
                temperature=0
                )

            answer = response.choices[0].message.content
            nr_tokens = response.usage.total_tokens
            return answer, nr_tokens

        except Exception as e:
            print(f'Exception: {e}')
            time.sleep(nr_retries * 2)

    raise Exception('Cannot query OpenAI model!')


In [ ]:
file_path = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/textanalysis/reviews.csv'

df = pd.read_csv(file_path)

nr_correct = 0
nr_tokens = 0

for _, row in df.iterrows():

    text = row['text']
    prompt = create_prompt(text)
    label, current_tokens = call_llm(prompt)

    ground_truth = row['sentiment']
    if label == ground_truth:
        nr_correct += 1
    nr_tokens += current_tokens

    print(f'Label: {label}; Ground truth: {ground_truth}')

print(f'Number of correct labels:\t{nr_correct}')
print(f'Number of tokens used   :\t{nr_tokens}')

Label: neg; Ground truth: neg
Label: neg; Ground truth: neg
Label: neg; Ground truth: neg
Label: neg; Ground truth: neg
Label: pos; Ground truth: pos
Label: pos; Ground truth: neg
Label: pos; Ground truth: neg
Label: neg; Ground truth: neg
Label: Negative; Ground truth: pos
Label: neg; Ground truth: neg
Number of correct labels:	7
Number of tokens used   :	1940
